### Year 1 Topology 1 Contingency Analysis Reporting - Lower Limit Violation

In [1]:
import pandas as pd
import numpy as np

#### Bus Voltage Data

In [2]:
list_volt = []
list_gens = [0,50,100]
list_lsc = ['lls','rls','hls']
for gen in list_gens:
    for lsc in list_lsc:
        file_out = 'savnw_sol_' + str(gen) +'_'+ lsc + '.xlsx'
        data_volt = pd.read_excel(file_out, sheet_name='Bus Voltage')
        data_v_nonan = data_volt.dropna(how='all').reset_index(drop=True)
        data_v_nonan['Scenario']= 'Solar = ' + str(gen) + ' MW, ' + lsc.upper()
        list_volt.append(data_v_nonan)
data_v = pd.concat(list_volt).reset_index(drop=True)
print(len(data_v))
display(data_v.head())

16700


,BUS,RECORD,TYPE,MIN/DROP,MAX/RISE,CONTINGENCY,BASE VOLTS,CONT VOLTS,DEVIATION,RANGE VIO,DEV VIO,Unnamed: 11,Scenario
0,101 NUC-A 21.600,ALL,RANGE,0.95,1.05,BASE CASE,1.020000,NaN,NaN,NaN,NaN,BASE VOLTS is pu base case voltage.,"Solar = 0 MW, LLS"
1,102 NUC-B 21.600,ALL,RANGE,0.95,1.05,BASE CASE,1.020000,NaN,NaN,NaN,NaN,CONT VOLTS is pu contingency case voltage.,"Solar = 0 MW, LLS"
2,103 SOLAR_PV 13.800,ALL,RANGE,0.95,1.05,BASE CASE,0.974748,NaN,NaN,NaN,NaN,DEVIATION is difference between contingency ca...,"Solar = 0 MW, LLS"
3,151 NUCPANT 500.00,ALL,RANGE,0.95,1.05,BASE CASE,1.021349,NaN,NaN,NaN,NaN,RANGE VIO is range violations calculated as bu...,"Solar = 0 MW, LLS"
4,152 MID500 500.00,ALL,RANGE,0.95,1.05,BASE CASE,1.045176,NaN,NaN,NaN,NaN,DEV VIO is deviation violations calculated as ...,"Solar = 0 MW, LLS"


#### Bus voltage data wrangling for LaTex reporting 

In [3]:
data_v = data_v.rename(columns={'BUS':'Bus', 
                                'CONTINGENCY':'Contingency',
                                'BASE VOLTS':'Base Voltage',                               
                                'CONT VOLTS':'Contingency Voltage',
                                'RANGE VIO' : 'Range Violation', 
                                'DEVIATION' : 'Deviation'})
data_v['Contingency'] = data_v['Contingency'].str.replace('&','\&')
data_v['Bus'] = data_v['Bus'].str.replace('_','\_')
data_v['Bus Number'] = data_v['Bus'].str.split(expand=True)[0]

#### Lower limit bus voltage violations 

In [4]:
data_low = data_v[data_v['Contingency Voltage']<0.9].reset_index(drop=True)
print(len(data_low))
display(data_low.head())

166


,Bus,RECORD,TYPE,MIN/DROP,MAX/RISE,Contingency,Base Voltage,Contingency Voltage,Deviation,Range Violation,DEV VIO,Unnamed: 11,Scenario,Bus Number
0,203 EAST230 230.00,ALL,RANGE,0.95,1.05,BUS 202,1.000144,0.891972,-0.108172,-0.058028,NaN,NaN,"Solar = 0 MW, LLS",203
1,203 EAST230 230.00,LIMIT,RANGE,0.90,1.10,BUS 202,1.000144,0.891972,-0.108172,-0.008028,NaN,NaN,"Solar = 0 MW, LLS",203
2,103 SOLAR\_PV 13.800,ALL,RANGE,0.95,1.05,BUS 205,0.974748,0.856086,-0.118662,-0.093914,NaN,NaN,"Solar = 0 MW, LLS",103
3,154 DOWNTN 230.00,ALL,RANGE,0.95,1.05,BUS 205,0.974748,0.856086,-0.118662,-0.093914,NaN,NaN,"Solar = 0 MW, LLS",154
4,3008 CATDOG 230.00,ALL,RANGE,0.95,1.05,BUS 205,0.993019,0.896238,-0.096782,-0.053762,NaN,NaN,"Solar = 0 MW, LLS",3008


#### Reporting of Buses with lower voltage violations

In [5]:
L_scen = list(data_low['Scenario'].unique())
print(f'It was seen that for Topology 1, all the studied scenarios reported lower voltage limit violation.')
print('------------------------------------------------------------------------------------------------------------------------------------------------------')
for vlscen in L_scen:
    print(vlscen)
    # Extracting bus data with voltage less than 0.9 PU
    data_lscen = data_low[data_low['Scenario'] == vlscen]
    print(f'For the studied scenario {vlscen}, the bus(es) reporting lower emergency range violation(s) are the buses',', '.join(data_lscen['Bus Number'].unique()),)
    # lower range violation latex table data reporting
    data_replv = data_lscen[['Bus Number','Contingency','Base Voltage', 'Contingency Voltage', 'Deviation', 'Range Violation']].reset_index(drop=True)
    
    unit_lowv = data_replv[data_replv['Contingency'].str.contains('UNIT')]
    if len(unit_lowv) != 0:
        unit_lv = unit_lowv.drop_duplicates(subset=['Bus Number'])
        for contlu in list(unit_lv['Contingency'].unique()):
            unit_cont = unit_lv[unit_lv['Contingency']==contlu]
            print(f'For the unit fault contingency {contlu}, the bus(es) reporting lower emergency range violation(s) are',', '.join(unit_cont['Bus Number'].unique()))
       
    
    bus_lowv = data_replv[data_replv['Contingency'].str.contains('BUS')]
    if len(bus_lowv) != 0:
        bus_lv = bus_lowv.drop_duplicates(subset=['Bus Number'])
        for contlb in list(bus_lv['Contingency'].unique()):
            bus_cont = bus_lv[bus_lv['Contingency']==contlb]
            print(f'For the bus fault contingency {contlb}, the bus(es) reporting lower emergency range violation(s) are',', '.join(bus_cont['Bus Number'].unique()))
        
    line_lowv = data_replv[data_replv['Contingency'].str.contains('SING OPN LIN')]
    if len(line_lowv) != 0:
        line_lv = line_lowv.drop_duplicates(subset=['Bus Number'])
        for contll in list(line_lv['Contingency'].unique()):
            line_cont = line_lv[line_lv['Contingency']==contll]
            print(f'For the single line fault contingency {contll}, the bus(es) reporting lower emergency range violation(s) are',', '.join(line_cont['Bus Number'].unique()))
    print('------------------------------------------------------------------------------------------------------------------------------------------------------')


It was seen that for Topology 1, all the studied scenarios reported lower voltage limit violation.
------------------------------------------------------------------------------------------------------------------------------------------------------
Solar = 0 MW, LLS
For the studied scenario Solar = 0 MW, LLS, the bus(es) reporting lower emergency range violation(s) are the buses 203, 103, 154, 3008
For the bus fault contingency BUS 202, the bus(es) reporting lower emergency range violation(s) are 203
For the bus fault contingency BUS 205, the bus(es) reporting lower emergency range violation(s) are 103, 154, 3008
------------------------------------------------------------------------------------------------------------------------------------------------------
Solar = 0 MW, RLS
For the studied scenario Solar = 0 MW, RLS, the bus(es) reporting lower emergency range violation(s) are the buses 103, 154, 203, 3007, 3008
For the bus fault contingency BUS 205, the bus(es) reporting lower e

Lower Voltage Limit Counts - Scenario 

In [6]:
vl_index = list(data_low['Scenario'].value_counts().index)
vl_counts = list(data_low['Scenario'].value_counts())
dict_vl_count = {
    'Scenario':vl_index,
    'Overload Counts':vl_counts
}
scen_lv_vc  = pd.DataFrame(dict_vl_count)
scen_lv_vc

,Scenario,Overload Counts
0,"Solar = 50 MW, HLS",32
1,"Solar = 100 MW, RLS",32
2,"Solar = 0 MW, HLS",24
3,"Solar = 50 MW, RLS",24
4,"Solar = 100 MW, HLS",22
5,"Solar = 0 MW, RLS",16
6,"Solar = 0 MW, LLS",8
7,"Solar = 50 MW, LLS",4
8,"Solar = 100 MW, LLS",4


Upper Voltage Limit Counts - Bus

In [7]:
vlb_index = list([i.strip().split()[0] for i in data_low['Bus'].value_counts().index])
vlb_counts = list(data_low['Bus'].value_counts())
dict_vlb_count = {
    'Bus':vlb_index,
    'Overload Counts':vlb_counts
}
bus_lv_vc  = pd.DataFrame(dict_vlb_count)
bus_lv_vc

,Bus,Overload Counts
0,103,38
1,154,38
2,203,26
3,3008,22
4,3007,14
5,205,14
6,153,6
7,204,4
8,3006,2
9,3018,2


Lower Voltage Limit Counts - Contingency

In [8]:
vlc_index = list(data_low['Contingency'].value_counts().index)
vlc_counts = list(data_low['Contingency'].value_counts())
dict_vlc_count = {
    'Contingency':vlc_index,
    'Overload Counts':vlc_counts
}
cont_lv_vc  = pd.DataFrame(dict_vlc_count)
cont_lv_vc

,Contingency,Overload Counts
0,BUS 205,68
1,SING OPN LIN 12 202-203(1),38
2,SING OPN LIN 10 201-205(\&1),22
3,SING OPN LIN 6 152-153(1),18
4,BUS 202,16
5,SING OPN LIN 9 201-202(1),4


Lower Voltage Result Summary - grouped by Scenario and Contingency 

In [9]:
data_fil_lv = data_low[['Scenario', 'Bus Number', 'Contingency']].drop_duplicates()
pivot_lv = pd.DataFrame(pd.pivot(data_fil_lv, index= ['Scenario','Contingency'], columns = 'Bus Number',values = 'Bus Number').to_records())
pivot_lv['Buses'] = pivot_lv['103'].astype(str).str.cat(pivot_lv[['153', '154', '203', '205', '3006', '3007', '3008']].astype(str), sep=',')
pivot_lv['Buses'] =pivot_lv['Buses'].str.replace(',nan','').str.replace('nan,','')
pivot_low = pivot_lv.drop(columns = list(data_fil_lv['Bus Number'].unique()))

In [10]:
pivot_table_lv = pd.DataFrame(pd.pivot_table(data_fil_lv, index= ['Scenario','Contingency'],values = 'Bus Number', aggfunc='count').to_records())
pivot_table_low = pivot_table_lv.rename(columns = {'Bus Number':'Bus Count'})
df_low = pd.merge(pivot_low, pivot_table_low, on=['Scenario','Contingency'], how='inner').sort_values(by='Bus Count', ascending=False).reset_index(drop=True)
df_low

,Scenario,Contingency,Buses,Bus Count
0,"Solar = 50 MW, HLS",BUS 205,"103,153,154,203,3006,3007,3008",8
1,"Solar = 100 MW, RLS",BUS 202,"103,154,203,205,3007,3008",7
2,"Solar = 0 MW, HLS",SING OPN LIN 12 202-203(1),"103,154,203,205,3008",6
3,"Solar = 50 MW, RLS",SING OPN LIN 10 201-205(\&1),"103,154,203,205,3007,3008",6
4,"Solar = 50 MW, RLS",BUS 205,"103,154,203,3007,3008",5
5,"Solar = 0 MW, RLS",BUS 205,"103,154,203,3007,3008",5
6,"Solar = 100 MW, HLS",BUS 205,"103,154,203,3007,3008",5
7,"Solar = 100 MW, RLS",SING OPN LIN 10 201-205(\&1),"103,154,203,205,3008",5
8,"Solar = 50 MW, HLS",SING OPN LIN 12 202-203(1),"103,154,203,205,3008",5
9,"Solar = 100 MW, HLS",SING OPN LIN 12 202-203(1),"103,154,203,205",4
